In [8]:

%env APP_DIR=/data/zerospeech/zrbench
from pathlib import Path

from zerospeech.benchmarks.sLM_21 import SLM21Submission

submission = SLM21Submission.load(Path('/data/zerospeech/zrbench/samples/sLM21-random-submission'))

env: APP_DIR=/data/zerospeech/zrbench


In [ ]:

%env APP_DIR=/data/zerospeech/zrbench
from pathlib import Path

from zerospeech.benchmarks.abx_LS import AbxLSSubmission

submission = AbxLSSubmission.load(Path('/data/zerospeech/zrbench/samples/abxLS-random-submission/'))

In [14]:
%env APP_DIR=/data/zerospeech/zrbench
submission.valid

env: APP_DIR=/data/zerospeech/zrbench


True

In [33]:
%env APP_DIR=/data/zerospeech/zrbench
from pathlib import Path
from zerospeech.benchmarks.sLM_21 import SLM21Submission
from zerospeech.benchmarks.sLM_21.validators import SLM21SubmissionValidator

submission = SLM21Submission.load(Path('/data/zerospeech/zrbench/samples/sLM21-random-submission'))

validator = SLM21SubmissionValidator()

res = validator.validate(submission)

env: APP_DIR=/data/zerospeech/zrbench


In [34]:
%env APP_DIR=/data/zerospeech/zrbench

for r in res:
    print(r)
#
# print('-'*20)
# print(f"{str(res[1])} -> {len(res[1].data)}")
# print(len(validator.dataset.index.subsets.lexical_dev.items.wav_list.files_list))
# print('-'*20)
# # print(res[1].data)

env: APP_DIR=/data/zerospeech/zrbench
ValidationOK(lexical_dev): dataframe validates tests
ValidationOK(lexical_dev): expected files found
ValidationOK(lexical_test): dataframe validates tests
ValidationOK(lexical_test): expected files found


In [10]:
import abc
from dataclasses import dataclass


class SubmissionBase(abc.ABC):

    @abc.abstractmethod
    def submission_valid(self, quiet: bool = False) -> bool:
        pass


@dataclass
class SubmissionX(SubmissionBase):
    a: int
    b: float
    c: str

    def submission_valid(self, quiet: bool = False) -> bool:
        condition = (self.a > 10) and (0 < self.b <= 20.5)
        if not quiet:
            if condition:
                print("submission is valid !!!")
            else:
                print("submission is not valid !!!")

        return condition


test = SubmissionX(a=1234, b=12.3, c="other")
test.submission_valid()

True

In [4]:
import abc
import functools
from pydantic import BaseModel

class SubmissionBase(BaseModel, abc.ABC):

    @abc.abstractmethod
    def submission_valid(self, quiet: bool = False) -> bool:
        pass


class SubmissionX(SubmissionBase):
    a: int
    b : float
    c: str

    @functools.lru_cache
    def submission_valid(self, quiet: bool = False) -> bool:
        condition = (self.a > 10) and (0 < self.b <= 20.5)
        if condition and not quiet:
            print("submission is not valid !!!")
        return condition

test = SubmissionX(a=1234, b=12.3, c="other")
test.submission_valid()

NameError: Field name "submission_valid" shadows a BaseModel attribute; use a different field name with "alias='submission_valid'".

In [14]:
import random
from typing import List


def length_comparison(dim: int):
    data = dict(ncols=[], col_len=1)

    def comparison(array: List[str]):
        data['ncols'].append(len(array))
        res = []

        if len(set(data['ncols'])) != 1:
            print(f"found error {len(set(data['ncols']))}")
            res = ['error']
        return res

    return comparison


test_1 = [[random.randint(1, 30) for _ in range(100)] for _ in range(50)]
test_2 = [[random.randint(1, 30) for _ in range(100)] for _ in range(50)]

# add two at random
test_2[29].append(random.randint(1, 30))
test_2[19].append(random.randint(1, 30))

comp1 = length_comparison(2)
comp2 = length_comparison(2)

results1 = []
results2 = []
for i1, i2 in zip(test_1, test_2):
    results1.extend(comp1(i1))
    results2.extend(comp2(i2))

print(results1)
print(results2)

found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
found error 2
[]
['error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error', 'error']
